# Reference
This Notebook uses the code provided by firefliesqn,Link Below
https://www.kaggle.com/firefliesqn/tabnet-cv-0-81
Thanks for providing the code firefliesqn

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing TabNet

In [ ]:
!pip install pytorch_tabnet 

# Importing Libraries

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score
np.random.seed(0)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.impute import SimpleImputer

import torch

from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

%matplotlib inline

# Data Preparation

In [ ]:
%%time
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

train['n_missing'] = train.isna().sum(axis=1)
test['n_missing'] = test.isna().sum(axis=1)
train['target'] = train['target'].astype(str)

features = [col for col in train.columns if col not in ['target', 'id']]

pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=200, output_distribution='normal'))
        ])
X = pipe.fit_transform(train[features])
X_test=pipe.transform(test[features])

In [ ]:
train.head()

# Modeling

In [ ]:
tabnet_params = dict(n_steps = 1,
                   optimizer_fn=torch.optim.Adam,
                   optimizer_params=dict(lr=1e-2, weight_decay = 5e-4),
                   scheduler_params={"step_size":1, # how to use learning rate scheduler
                                     "gamma":0.9},
                   scheduler_fn=torch.optim.lr_scheduler.StepLR,
                   mask_type='entmax',
                   verbose = 5)

In [ ]:
from sklearn.model_selection import KFold
import torch

kf = KFold(n_splits=5, random_state = 42, shuffle = True)
preds = np.zeros((540000,))
for  fold , (train_index, test_index) in enumerate(kf.split(X)):
    print(20*"*")
    print("Fold {}:".format(fold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = train.target[train_index].values, train.target[test_index].values

    clf = TabNetClassifier(**tabnet_params)
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs= 10, patience=5,
        batch_size=1024*10, virtual_batch_size=128*10,
        num_workers=0,
        weights=1,
        drop_last=False
    ) 
    preds += clf.predict_proba(X_test)[:,1]/5
    print(preds.shape)

# Submission

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sub.iloc[:,1]= preds
sub=sub.set_index('id')
sub.to_csv('submission.csv')
sub.head()